# An Example 2D optimisation
## Inspired by the following notebook:
[scikit-optimize benchmarks](https://github.com/scikit-optimize/scikit-optimize.github.io/blob/master/notebooks/strategy-comparison.md)

The [Branin-Hoo function](https://www.sfu.ca/~ssurjano/branin.html) is a commonly used function for benchmarking black-box global optimisation algorithms

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

In [ ]:
# local modules
import sys; sys.path.append('..')
import optimisation as op
from optimisation import plot3D
from utils import synthetic_data

In [ ]:
# Make deterministic
np.random.seed(42)

# Target Function

In [ ]:
x = np.linspace(-5, 10, 100)
y = np.linspace(0, 15, 100)
X, Y = np.meshgrid(x, y)
points = np.vstack((X.ravel(), Y.ravel())).T

from skopt.benchmarks import branin as _branin

@np.vectorize
def branin(x, y):
    return _branin((x,y)) + 5 * np.random.randn()

Z = branin(X, Y)
best_z = np.min(Z)

class TestEvaluator(op.Evaluator):
    def test_config(self, config):
        return branin(config.x, config.y)
evaluator = TestEvaluator()

In [ ]:
plot3D.surface3D(X, Y, Z)

# Helper Functions

In [ ]:
Surrogate = op.SciKitGPSurrogate.Custom(gp_params={
    'alpha': 1e-10, # default noise level, deal with noise with the white kernel
    'kernel': 1.0 * gp.kernels.Matern(nu=2.5) + gp.kernels.WhiteKernel(),
    #'kernel': gp.kernels.ConstantKernel() + 1.0 * gp.kernels.Matern(nu=2.5) + gp.kernels.WhiteKernel(),
    'n_restarts_optimizer': 10
})
strategy = op.AcquisitionStrategy(
    pre_phase_steps = 6,
    acquisition_function = ('UCB', {'kappa' : 4}), #('EI', {'xi' : 0.1}),
    #parallel_strategy = 'kb'
    #parallel_strategy = 'mc'
)
ranges = {'x' : x, 'y' : y}

bo = op.BayesianOptimisationOptimiser(
    ranges, maximise_cost=False,
    acquisition_strategy=strategy,
    Surrogate=Surrogate, maximisation_args=None, close_tolerance=1e-4)

In [ ]:
op.gui.optimiser_progress_bar(bo)
bo.run_sequential(evaluator, max_jobs=30)
# run the multithreaded version to try out the parallel strategies
#bo.run_multithreaded(evaluators, max_jobs=60)

# or run interactively to see the log as it runs
#task = lambda: bo.run_sequential(evaluator, max_jobs=60)
#op.gui.interactive(bo, task)

In [ ]:
bo.plot_cost_over_time(true_best=best_z);

In [ ]:
def plot(n, step):
    bo.plot_step_1D('x', n, sur_through_all=True)
op.gui.step_log_slider(bo, plot);

In [ ]:
def plot2(n, step):
    bo.plot_step_1D('y', n, sur_through_all=True)
op.gui.step_log_slider(bo, plot2);

In [ ]:
num_random = bo.num_randomly_chosen()
num_bayes = len(bo.samples) - num_random
print('number of random samples: {}'.format(num_random))
print('number of bayes samples: {}'.format(num_bayes))

In [ ]:
def plot2D(n, step):
    bo.plot_step_2D('x', 'y', n, true_cost=Z)
op.gui.step_log_slider(bo, plot2D, pre_compute=False);

In [ ]:
bo.scatter_plot('x', 'y')

# Try optimising the same function with random search

In [ ]:
ra = op.RandomSearchOptimiser(ranges, maximise_cost=False)
ra.run_sequential(evaluator, max_jobs=1000)

In [ ]:
ra.plot_cost_over_time(true_best=best_z);

In [ ]:
ra.scatter_plot('x', 'y')